So I always wondered how instagram keeps showing me some reels that are very similar to the ones I already watched to keep me engaged to it. Same with youtube. And flipkart for products...

So I wanted to try and make one for books.

There are few different types in it.
- popularity based: everyone gets the same top product reccomendations
- content based: based on what you're looking at or liked so far
- collaborative: based on what - the people who watched the same content you did -are watching...
- hybrid: mix up of those above.

First, let's get the data for books. I found one that contains data of around 10k books.

I found it here, https://github.com/malcolmosh/goodbooks-10k-extended?tab=readme-ov-file

In [ ]:
import pandas as pd

from ast import literal_eval

books_df = pd.read_csv('https://raw.githubusercontent.com/malcolmosh/goodbooks-10k/master/books_enriched.csv', index_col=[1], converters={"genres": literal_eval}).drop(columns=["Unnamed: 0"])

In [ ]:
#take a look at the data
books_df.head()

,authors,average_rating,best_book_id,book_id,books_count,description,genres,goodreads_book_id,image_url,isbn,...,ratings_3,ratings_4,ratings_5,ratings_count,small_image_url,title,work_id,work_ratings_count,work_text_reviews_count,authors_2
index,,,,,,,,,,,,,,,,,,,,,
0,['Suzanne Collins'],4.34,2767052,1,272,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"[young-adult, fiction, fantasy, science-fictio...",2767052,https://images.gr-assets.com/books/1447303603m...,439023483,...,560092,1481305,2706317,4780653,https://images.gr-assets.com/books/1447303603s...,"The Hunger Games (The Hunger Games, #1)",2792775,4942365,155254,['Suzanne Collins']
1,"['J.K. Rowling', 'Mary GrandPré']",4.44,3,2,491,Harry Potter's life is miserable. His parents ...,"[fantasy, fiction, young-adult, classics]",3,https://images.gr-assets.com/books/1474154022m...,439554934,...,455024,1156318,3011543,4602479,https://images.gr-assets.com/books/1474154022s...,Harry Potter and the Sorcerer's Stone (Harry P...,4640799,4800065,75867,"['J.K. Rowling', 'Mary GrandPré']"
2,['Stephenie Meyer'],3.57,41865,3,226,About three things I was absolutely positive.\...,"[young-adult, fantasy, romance, fiction, paran...",41865,https://images.gr-assets.com/books/1361039443m...,316015849,...,793319,875073,1355439,3866839,https://images.gr-assets.com/books/1361039443s...,"Twilight (Twilight, #1)",3212258,3916824,95009,['Stephenie Meyer']
3,['Harper Lee'],4.25,2657,4,487,The unforgettable novel of a childhood in a sl...,"[classics, fiction, historical-fiction, young-...",2657,https://images.gr-assets.com/books/1361975680m...,61120081,...,446835,1001952,1714267,3198671,https://images.gr-assets.com/books/1361975680s...,To Kill a Mockingbird,3275794,3340896,72586,['Harper Lee']
4,['F. Scott Fitzgerald'],3.89,4671,5,1356,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"[classics, fiction, historical-fiction, romance]",4671,https://images.gr-assets.com/books/1490528560m...,743273567,...,606158,936012,947718,2683664,https://images.gr-assets.com/books/1490528560s...,The Great Gatsby,245494,2773745,51992,['F. Scott Fitzgerald']


In [ ]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 9999
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   authors                    10000 non-null  object 
 1   average_rating             10000 non-null  float64
 2   best_book_id               10000 non-null  int64  
 3   book_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   description                9943 non-null   object 
 6   genres                     10000 non-null  object 
 7   goodreads_book_id          10000 non-null  int64  
 8   image_url                  10000 non-null  object 
 9   isbn                       9300 non-null   object 
 10  isbn13                     9415 non-null   float64
 11  language_code              10000 non-null  object 
 12  original_publication_year  9979 non-null   float64
 13  original_title             9415 non-null   object 
 

It has lot of info related to books.

But lets check the data that is needed by us.

In [ ]:
data_mini  = books_df[['title','authors','average_rating','ratings_count','genres','description','image_url']]

In [ ]:
data_mini.head()

,title,authors,average_rating,ratings_count,genres,description,image_url
0,"The Hunger Games (The Hunger Games, #1)",['Suzanne Collins'],4.34,4780653,"[young-adult, fiction, fantasy, science-fictio...",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,https://images.gr-assets.com/books/1447303603m...
1,Harry Potter and the Sorcerer's Stone (Harry P...,"['J.K. Rowling', 'Mary GrandPré']",4.44,4602479,"[fantasy, fiction, young-adult, classics]",Harry Potter's life is miserable. His parents ...,https://images.gr-assets.com/books/1474154022m...
2,"Twilight (Twilight, #1)",['Stephenie Meyer'],3.57,3866839,"[young-adult, fantasy, romance, fiction, paran...",About three things I was absolutely positive.\...,https://images.gr-assets.com/books/1361039443m...
3,To Kill a Mockingbird,['Harper Lee'],4.25,3198671,"[classics, fiction, historical-fiction, young-...",The unforgettable novel of a childhood in a sl...,https://images.gr-assets.com/books/1361975680m...
4,The Great Gatsby,['F. Scott Fitzgerald'],3.89,2683664,"[classics, fiction, historical-fiction, romance]",Alternate Cover Edition ISBN: 0743273567 (ISBN...,https://images.gr-assets.com/books/1490528560m...


In [ ]:
data_mini.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           10000 non-null  object 
 1   authors         10000 non-null  object 
 2   average_rating  10000 non-null  float64
 3   ratings_count   10000 non-null  int64  
 4   genres          10000 non-null  object 
 5   description     9943 non-null   object 
 6   image_url       10000 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 625.0+ KB


In [ ]:

data_mini.sort_values(by='average_rating',ascending=False).head(15)



,title,authors,average_rating,ratings_count,genres,description,image_url
index,,,,,,,
3353,The Complete Calvin and Hobbes,['Bill Watterson'],4.82,28900,"[comics, graphic-novels, fiction, classics]",[ \n Box Set\n | Book One | Book Two | Book T...,https://images.gr-assets.com/books/1473064526m...
810,"Words of Radiance (The Stormlight Archive, #2)",['Brandon Sanderson'],4.77,73572,"[fantasy, fiction]","Words of Radiance, Book Two of the Stormlight ...",https://images.gr-assets.com/books/1391535251m...
3043,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...","['J.K. Rowling', 'Mary GrandPré']",4.77,33220,"[fantasy, young-adult, fiction, classics]",Box Set containing Harry Potter and the Sorcer...,https://s.gr-assets.com/assets/nophoto/book/11...
6779,ESV Study Bible,"['Anonymous', 'Lane T. Dennis', 'Wayne A. Grud...",4.76,8953,"[christian, nonfiction, religion, classics, hi...",The ESV Study Bible was designed to help you u...,https://images.gr-assets.com/books/1410151002m...
7403,Mark of the Lion Trilogy,['Francine Rivers'],4.76,9081,"[christian, historical-fiction, fiction, roman...",This best-selling trilogy chronicles a tale of...,https://images.gr-assets.com/books/1349032180m...
4068,It's a Magical World: A Calvin and Hobbes Coll...,['Bill Watterson'],4.75,22351,"[comics, graphic-novels, fiction]",When cartoonist Bill Watterson announced that ...,https://images.gr-assets.com/books/1437420710m...
387,"Harry Potter Boxset (Harry Potter, #1-7)",['J.K. Rowling'],4.74,190050,"[fantasy, young-adult, fiction, classics]","Over 4000 pages of Harry Potter and his world,...",https://images.gr-assets.com/books/1392579059m...
5593,There's Treasure Everywhere: A Calvin and Hobb...,['Bill Watterson'],4.74,16766,"[comics, fiction, graphic-novels]",In the world that Calvin and his stuffed tiger...,https://s.gr-assets.com/assets/nophoto/book/11...
3457,"Harry Potter Collection (Harry Potter, #1-6)",['J.K. Rowling'],4.73,24618,"[fantasy, fiction, young-adult, classics]","Six years of magic, adventure, and mystery mak...",https://images.gr-assets.com/books/1328867351m...


so we saw those with lots of ratings and those with high ratings.
What if to capture popularity, we choose a mix of both average_rating and ratings_count?

to do that, we can have another metric 'popularity' that is 0.5*average_rating + 0.5*ratings_count/max(ratings_count)



In [ ]:
max_ratings_count = max(data_mini['ratings_count'])
min_ratings_count = min(data_mini['ratings_count'])
max_rating = max(data_mini['average_rating'])
min_rating = min(data_mini['average_rating'])

print(max_ratings_count)
print(min_ratings_count)
print(max_rating)
print(min_rating)

data_mini['popularity'] = 0.5*(data_mini['average_rating']-min_rating)/(max_rating-min_rating) + 0.5*(data_mini['ratings_count']-min_ratings_count)/(max_ratings_count-min_ratings_count)

4780653
2716
4.82
2.47


/tmp/ipython-input-3060054290.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mini['popularity'] = 0.5*(data_mini['average_rating']-min_rating)/(max_rating-min_rating) + 0.5*(data_mini['ratings_count']-min_ratings_count)/(max_ratings_count-min_ratings_count)


In [ ]:
top_rated = data_mini.sort_values(by ='popularity',ascending = False)

In [ ]:
top_rated.head(15)

,title,authors,average_rating,ratings_count,genres,description,image_url,popularity
index,,,,,,,,
1,Harry Potter and the Sorcerer's Stone (Harry P...,"['J.K. Rowling', 'Mary GrandPré']",4.44,4602479,"[fantasy, fiction, young-adult, classics]",Harry Potter's life is miserable. His parents ...,https://images.gr-assets.com/books/1474154022m...,0.900503
0,"The Hunger Games (The Hunger Games, #1)",['Suzanne Collins'],4.34,4780653,"[young-adult, fiction, fantasy, science-fictio...",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,https://images.gr-assets.com/books/1447303603m...,0.897872
3,To Kill a Mockingbird,['Harper Lee'],4.25,3198671,"[classics, fiction, historical-fiction, young-...",The unforgettable novel of a childhood in a sl...,https://images.gr-assets.com/books/1361975680m...,0.713173
2,"Twilight (Twilight, #1)",['Stephenie Meyer'],3.57,3866839,"[young-adult, fantasy, romance, fiction, paran...",About three things I was absolutely positive.\...,https://images.gr-assets.com/books/1361039443m...,0.638414
21,Harry Potter and the Deathly Hallows (Harry Po...,"['J.K. Rowling', 'Mary GrandPré']",4.61,1746574,"[fantasy, young-adult, fiction, classics]",Harry Potter is leaving Privet Drive for the l...,https://images.gr-assets.com/books/1474171184m...,0.637810
15,Harry Potter and the Prisoner of Azkaban (Harr...,"['J.K. Rowling', 'Mary GrandPré', 'Rufus Beck']",4.53,1832823,"[fantasy, young-adult, fiction, classics]",Harry Potter's third year at Hogwarts is full ...,https://images.gr-assets.com/books/1499277281m...,0.629814
5,The Fault in Our Stars,['John Green'],4.26,2346404,"[young-adult, romance, fiction, contemporary]",Despite the tumor-shrinking medical miracle th...,https://images.gr-assets.com/books/1360206420m...,0.626113
20,Harry Potter and the Goblet of Fire (Harry Pot...,"['J.K. Rowling', 'Mary GrandPré']",4.53,1753043,"[fantasy, young-adult, fiction, classics]",Harry Potter is midway through his training as...,https://images.gr-assets.com/books/1361482611m...,0.621466
23,Harry Potter and the Half-Blood Prince (Harry ...,"['J.K. Rowling', 'Mary GrandPré']",4.54,1678823,"[fantasy, young-adult, fiction, classics]",The war against Voldemort is not going well; e...,https://images.gr-assets.com/books/1361039191m...,0.615826


In [ ]:
book_recs_by_popularity = top_rated.head(15)[['title','authors']]
book_recs_by_popularity

,title,authors
index,,
1,Harry Potter and the Sorcerer's Stone (Harry P...,"['J.K. Rowling', 'Mary GrandPré']"
0,"The Hunger Games (The Hunger Games, #1)",['Suzanne Collins']
3,To Kill a Mockingbird,['Harper Lee']
2,"Twilight (Twilight, #1)",['Stephenie Meyer']
21,Harry Potter and the Deathly Hallows (Harry Po...,"['J.K. Rowling', 'Mary GrandPré']"
15,Harry Potter and the Prisoner of Azkaban (Harr...,"['J.K. Rowling', 'Mary GrandPré', 'Rufus Beck']"
5,The Fault in Our Stars,['John Green']
20,Harry Potter and the Goblet of Fire (Harry Pot...,"['J.K. Rowling', 'Mary GrandPré']"
23,Harry Potter and the Half-Blood Prince (Harry ...,"['J.K. Rowling', 'Mary GrandPré']"


Voila! That's right there is the popularity based recommendation.

Let's look for content based on next. You input what you like and you get recommened what you might like based on that.

In [ ]:
data_mini[data_mini['description'].isnull()]

,title,authors,average_rating,ratings_count,genres,description,image_url
427,Holy Bible: King James Version,['Anonymous'],4.43,159457,"[religion, classics, nonfiction, christian, hi...",NaN,https://images.gr-assets.com/books/1313518530m...
911,Journey to the Center of the Earth (Extraordin...,['Jules Verne'],3.84,89410,"[classics, science-fiction, fiction, fantasy]",NaN,https://s.gr-assets.com/assets/nophoto/book/11...
1044,Aesop's Fables,"['Aesop', 'Laura Harris', 'Laura Gibbs']",4.05,88508,"[classics, fiction, fantasy, philosophy]",NaN,https://s.gr-assets.com/assets/nophoto/book/11...
1244,Howl and Other Poems,"['Allen Ginsberg', 'William Carlos Williams']",4.14,71968,"[poetry, classics, fiction]",NaN,https://images.gr-assets.com/books/1327870926m...
1252,The Book of Mormon: Another Testament of Jesus...,"['Anonymous', 'Joseph Smith Jr.']",4.37,63530,"[religion, nonfiction, spirituality, history, ...",NaN,https://images.gr-assets.com/books/1327389004m...
1566,Shakespeare's Sonnets,"['William Shakespeare', 'Katherine Duncan-Jones']",4.24,58291,"[poetry, classics, fiction, romance]",NaN,https://s.gr-assets.com/assets/nophoto/book/11...
1579,Leaves of Grass,['Walt Whitman'],4.12,57763,"[poetry, classics, fiction, philosophy]",NaN,https://images.gr-assets.com/books/1418012204m...
1638,The Bonfire of the Vanities,['Tom Wolfe'],3.81,52062,"[fiction, classics, contemporary]",NaN,https://s.gr-assets.com/assets/nophoto/book/11...
2502,The Autobiography of Benjamin Franklin,"['Benjamin Franklin', 'Lewis Leary']",3.83,33061,"[biography, history, nonfiction, classics, mem...",NaN,https://s.gr-assets.com/assets/nophoto/book/11...
3165,First Things First,"['Stephen R. Covey', 'A. Roger Merrill', 'Rebe...",4.06,26813,"[self-help, business, nonfiction, psychology, ...",NaN,https://s.gr-assets.com/assets/nophoto/book/11...


In [ ]:
data_mini[data_mini['description'].isnull()].fillna("") # filling the nan values with empty string.

,title,authors,average_rating,ratings_count,genres,description,image_url,popularity
index,,,,,,,,
427,Holy Bible: King James Version,['Anonymous'],4.43,159457,"[religion, classics, nonfiction, christian, hi...",,https://images.gr-assets.com/books/1313518530m...,0.433424
911,Journey to the Center of the Earth (Extraordin...,['Jules Verne'],3.84,89410,"[classics, science-fiction, fiction, fantasy]",,https://s.gr-assets.com/assets/nophoto/book/11...,0.300562
1044,Aesop's Fables,"['Aesop', 'Laura Harris', 'Laura Gibbs']",4.05,88508,"[classics, fiction, fantasy, philosophy]",,https://s.gr-assets.com/assets/nophoto/book/11...,0.345148
1244,Howl and Other Poems,"['Allen Ginsberg', 'William Carlos Williams']",4.14,71968,"[poetry, classics, fiction]",,https://images.gr-assets.com/books/1327870926m...,0.362566
1252,The Book of Mormon: Another Testament of Jesus...,"['Anonymous', 'Joseph Smith Jr.']",4.37,63530,"[religion, nonfiction, spirituality, history, ...",,https://images.gr-assets.com/books/1327389004m...,0.410619
1566,Shakespeare's Sonnets,"['William Shakespeare', 'Katherine Duncan-Jones']",4.24,58291,"[poetry, classics, fiction, romance]",,https://s.gr-assets.com/assets/nophoto/book/11...,0.382412
1579,Leaves of Grass,['Walt Whitman'],4.12,57763,"[poetry, classics, fiction, philosophy]",,https://images.gr-assets.com/books/1418012204m...,0.356824
1638,The Bonfire of the Vanities,['Tom Wolfe'],3.81,52062,"[fiction, classics, contemporary]",,https://s.gr-assets.com/assets/nophoto/book/11...,0.290270
2502,The Autobiography of Benjamin Franklin,"['Benjamin Franklin', 'Lewis Leary']",3.83,33061,"[biography, history, nonfiction, classics, mem...",,https://s.gr-assets.com/assets/nophoto/book/11...,0.292537


In [ ]:
#look at all the genres

set(data_mini['genres'].sum()),len(set(data_mini['genres'].sum()))

({'art',
  'biography',
  'books',
  'business',
  'chick-lit',
  'christian',
  'classics',
  'comics',
  'contemporary',
  'cookbooks',
  'crime',
  'fantasy',
  'fiction',
  'gay-and-lesbian',
  'graphic-novels',
  'historical-fiction',
  'history',
  'horror',
  'humor-and-comedy',
  'manga',
  'memoir',
  'music',
  'mystery',
  'nonfiction',
  'paranormal',
  'philosophy',
  'poetry',
  'psychology',
  'religion',
  'romance',
  'science',
  'science-fiction',
  'self-help',
  'spirituality',
  'sports',
  'suspense',
  'thriller',
  'travel',
  'young-adult'},
 39)

In [ ]:
#look at all the authors
#need a little change here

set(data_mini['authors']),len(set(data_mini['authors']))

({"['Beatrice Sparks', 'Anonymous']",
  "['Jussi Adler-Olsen']",
  "['Chris Heimerdinger']",
  "['Anthony Doerr']",
  "['[Katharine Hepburn]']",
  "['Michael Buckley', 'Peter Ferguson']",
  "['Jennifer E. Smith']",
  "['Edward Said']",
  "['Marguerite Henry', 'Wesley Dennis']",
  "['Maxine Hong Kingston']",
  "['Richard Dawkins']",
  "['[Guillermo del Toro', 'Chuck Hogan]']",
  "['Frank Miller']",
  "['Kate Douglas Wiggin']",
  "['Shane Claiborne']",
  "['[Tom Stoppard]']",
  "['[Jennifer L. Armentrout]']",
  "['K.A. Linde']",
  "['Lev Grossman']",
  "['Koren Zailckas']",
  "['Scott Cunningham']",
  "['Laura Ingalls Wilder', 'Garth Williams']",
  "['Natalie Goldberg']",
  "['[Agatha Christie]']",
  "['[Irving Shulman]']",
  "['Anita Amirrezvani']",
  "['Amin Maalouf', 'Russell Harris']",
  "['Sabahattin Ali']",
  "['[Elizabeth   Hunter]']",
  "['Shiv Khera']",
  "['Anne Frank', 'Eleanor Roosevelt', 'B.M. Mooyaart-Doubleday']",
  "['Sherwood Anderson']",
  "['Katie Klein']",
  "['Alexan

In [ ]:
#look at all the authors of a particular index

print(data_mini['authors'].iloc[9997].tolist())

AttributeError: 'str' object has no attribute 'tolist'

NOw to convert the features into vectors, we use tf-idf for description, mhe for authors and genres.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, normalize
from scipy.sparse import hstack

In [ ]:
#description
tfidf_desc = TfidfVectorizer(stop_words="english", max_features=5000)
desc_matrix = tfidf_desc.fit_transform(data_mini["description"].fillna(""))


In [ ]:
desc_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 508339 stored elements and shape (10000, 5000)>

In [ ]:
#multi hot for genres
mlb1 = MultiLabelBinarizer()
genre_matrix = mlb1.fit_transform(data_mini["genres"])


In [ ]:
genre_matrix.shape

(10000, 39)

In [ ]:
mlb2 = MultiLabelBinarizer()
author_matrix = mlb2.fit_transform(data_mini["authors"])


In [ ]:
author_matrix.shape

(10000, 254)

In [ ]:
# Sparse matrix stacking (keeps memory efficient)
combined_matrix = hstack([desc_matrix, genre_matrix, author_matrix])

# Normalize for cosine similarity
combined_matrix = normalize(combined_matrix)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(combined_matrix)


In [ ]:

# Example: books most similar to Book A
book_idx = 2
similar_scores = list(enumerate(similarity[book_idx]))
sorted_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)

print("Recommendations for:", data_mini['title'][book_idx], "genre: ",data_mini['genres'][book_idx])
for idx, score in sorted_scores[1:50]:
    print(f"  {data_mini['title'][idx]} (score={score:.2f})","genre: ",data_mini['genres'][idx])


Recommendations for: Twilight (Twilight, #1) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  The Twilight Saga Complete Collection  (Twilight, #1-4 + 3.5) (score=0.97) genre:  ['fantasy', 'young-adult', 'romance', 'paranormal', 'fiction']
  The Twilight Collection (Twilight, #1-3) (score=0.96) genre:  ['young-adult', 'fantasy', 'romance', 'paranormal', 'fiction']
  New Moon (Twilight, #2) (score=0.96) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  Eclipse (Twilight, #3) (score=0.96) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  Breaking Dawn (Twilight, #4) (score=0.96) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  The Twilight Saga: The Official Illustrated Guide (Twilight, #4.5) (score=0.96) genre:  ['young-adult', 'fantasy', 'paranormal', 'fiction', 'romance']
  The Short Second Life of Bree Tanner: An Eclipse Novella (Twilight, #3.5) (score=0.96) genre:  ['young-adult', 'fantasy'

So, this works fine. We can see that books similar to the one we highlighted are shown.


There is sBERT, a transformer that can have better representation of the description than using word2vec. So let's use it.

In [ ]:
from sentence_transformers import SentenceTransformer

# load SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
desc_embeds = model.encode(data_mini["description"].fillna(""))

In [ ]:
#next time, use this. It's good to see the progress

desc_embeds = model.encode(
    data_mini["description"].fillna("").tolist(),
    batch_size=32,      # tune: 16, 32, 64 depending on RAM
    show_progress_bar=True
)


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [ ]:
#let's save the data_mini
#don't forget to mount the drive

data_mini.to_csv("/content/drive/MyDrive/data science practice/book_rec_system/data_mini_books.csv",index=False)

In [ ]:
import numpy as np

'''# Save
np.save("/content/drive/MyDrive/data science practice/book_rec_system/desc_embeds.npy", desc_embeds)'''

# Load later
desc_embeds = np.load("/content/drive/MyDrive/data science practice/book_rec_system/desc_embeds.npy")

In [ ]:
desc_embeds.shape, genre_matrix.shape, author_matrix.shape

((10000, 384), (10000, 39), (10000, 254))

# similarity using sentence transformer 1


> Add blockquote



In [ ]:
#multi hot for genres
mlb1 = MultiLabelBinarizer()
genre_matrix = mlb1.fit_transform(data_mini["genres"])

mlb2 = MultiLabelBinarizer()
author_matrix = mlb2.fit_transform(data_mini["authors"])

desc_embeds = np.load("/content/drive/MyDrive/data science practice/book_rec_system/desc_embeds.npy")

from scipy.sparse import csr_matrix, hstack

desc_sparse = csr_matrix(desc_embeds)   # convert dense SBERT embeddings to sparse

# Sparse matrix stacking (keeps memory efficient)
combined_matrix2 = hstack([desc_sparse, genre_matrix, author_matrix])

# Normalize for cosine similarity
combined_matrix2 = normalize(combined_matrix2)

book_ids = [121,36,44]

#get average of the book_ids embeds
avg_book_embeds = sum(desc_embeds[book_ids])/len(book_ids)
avg_book_embeds.reshape(1,-1)
print(avg_book_embeds.shape)



similarity3 = cosine_similarity(desc_embeds, avg_book_embeds.reshape(1, -1))
#print(similarity3.shape)
similarity3_list = list(enumerate(similarity3))
sorted_similarity = sorted(similarity3_list,key = lambda x:x[1],reverse = True)

to_choose = 50
to_show = 15

#print the loved books
print("loved books")
for book_id in book_ids:
  print(data_mini.iloc[book_id][['title','authors','genres']])


rec_df = pd.DataFrame(columns=['title','authors','genres','average_rating','similarity_score'])


for idx,score in sorted_similarity[1:to_choose]:
  if idx not in book_ids:
    row = data_mini.iloc[[idx]][['title','authors','genres','average_rating']]
    row['similarity_score'] = score
    rec_df = pd.concat([rec_df,row],ignore_index = True)

rec_df = rec_df.sort_values(by='average_rating',ascending=False)

print("recommended books")

'''for i in range(to_show):
  print(list(rec_df.iloc[i][['title','authors','genres','average_rating']]))'''

alpha = 0.6 #giving weight to the rating

rec_df['final_score'] = rec_df['average_rating']*alpha + rec_df['similarity_score']*(1-alpha)
rec_df.sort_values(by = 'final_score',ascending = False)




(384,)
loved books
title      A Storm of Swords (A Song of Ice and Fire, #3)
authors                            ['George R.R. Martin']
genres                                 [fantasy, fiction]
Name: 121, dtype: object
title      The Lightning Thief (Percy Jackson and the Oly...
authors                                     ['Rick Riordan']
genres                       [fantasy, young-adult, fiction]
Name: 36, dtype: object
title             City of Bones (The Mortal Instruments, #1)
authors                                  ['Cassandra Clare']
genres     [fantasy, young-adult, paranormal, romance, fi...
Name: 44, dtype: object
recommended books


/tmp/ipython-input-1970105724.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rec_df = pd.concat([rec_df,row],ignore_index = True)


,title,authors,genres,average_rating,similarity_score,final_score
39,The Mortal Instruments (The Mortal Instruments...,['Cassandra Clare'],"[fantasy, young-adult, romance, paranormal, fi...",4.59,0.556636,2.976654
25,Percy Jackson and the Olympians (Percy Jackson...,['Rick Riordan'],"[fantasy, young-adult, fiction]",4.58,0.566821,2.974728
9,The Mortal Instruments Boxed Set: City of Bone...,['Cassandra Clare'],"[fantasy, young-adult, paranormal, romance, fi...",4.57,0.581413,2.974565
6,A Game of Thrones / A Clash of Kings (A Song o...,['George R.R. Martin'],"[fantasy, fiction, science-fiction]",4.53,0.590508,2.954203
5,"City of Heavenly Fire (The Mortal Instruments,...",['Cassandra Clare'],"[fantasy, young-adult, romance, paranormal, fi...",4.48,0.592823,2.925129
37,"Shadowfever (Fever, #5)",['Karen Marie Moning'],"[fantasy, paranormal, romance, fiction]",4.46,0.557285,2.898914
35,"A Game of Thrones (A Song of Ice and Fire, #1)",['George R.R. Martin'],"[fantasy, fiction]",4.45,0.557639,2.893055
14,"The Witch With No Name (The Hollows, #13)",['Kim Harrison'],"[fantasy, paranormal, fiction, romance]",4.43,0.571388,2.886555
0,"Faefever (Fever, #3)",['Karen Marie Moning'],"[paranormal, fantasy, romance]",4.34,0.644515,2.861806
44,"A Clash of Kings (A Song of Ice and Fire, #2)",['George R.R. Martin'],"[fantasy, fiction]",4.40,0.553231,2.861293


In [ ]:
from scipy.sparse import csr_matrix, hstack

desc_sparse = csr_matrix(desc_embeds)   # convert dense SBERT embeddings to sparse

# Sparse matrix stacking (keeps memory efficient)
combined_matrix2 = hstack([desc_sparse, genre_matrix, author_matrix])

# Normalize for cosine similarity
combined_matrix2 = normalize(combined_matrix2)

In [ ]:
combined_matrix2.shape

(10000, 677)

In [ ]:
similarity2 = cosine_similarity(combined_matrix2)


In [ ]:

# Example: books most similar to Book A
book_idx = 2
similar_scores = list(enumerate(similarity2[book_idx]))
sorted_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)

print("Recommendations for:", data_mini['title'][book_idx], "genre: ",data_mini['genres'][book_idx])
for idx, score in sorted_scores[1:50]:
    print(f"  {data_mini['title'][idx]} (score={score:.2f})","genre: ",data_mini['genres'][idx])

Recommendations for: Twilight (Twilight, #1) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  The Twilight Saga Complete Collection  (Twilight, #1-4 + 3.5) (score=0.98) genre:  ['fantasy', 'young-adult', 'romance', 'paranormal', 'fiction']
  Eclipse (Twilight, #3) (score=0.98) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  The Twilight Collection (Twilight, #1-3) (score=0.98) genre:  ['young-adult', 'fantasy', 'romance', 'paranormal', 'fiction']
  Breaking Dawn (Twilight, #4) (score=0.98) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']
  Midnight Sun (Twilight, #1.5) (score=0.98) genre:  ['young-adult', 'fantasy', 'romance', 'paranormal', 'fiction']
  The Short Second Life of Bree Tanner: An Eclipse Novella (Twilight, #3.5) (score=0.98) genre:  ['young-adult', 'fantasy', 'paranormal', 'fiction', 'romance']
  New Moon (Twilight, #2) (score=0.98) genre:  ['young-adult', 'fantasy', 'romance', 'fiction', 'paranormal']

In [ ]:
book_ids = [121]

In [ ]:


#get average of the book_ids embeds
avg_book_embeds = sum(desc_embeds[book_ids])/len(book_ids)
avg_book_embeds.reshape(1,-1)
print(avg_book_embeds.shape)



similarity3 = cosine_similarity(desc_embeds, avg_book_embeds.reshape(1, -1))
#print(similarity3.shape)
similarity3_list = list(enumerate(similarity3))
sorted_similarity = sorted(similarity3_list,key = lambda x:x[1],reverse = True)

to_choose = 50
to_show = 15

#print the loved books
print("loved books")
for book_id in book_ids:
  print(data_mini.iloc[book_id][['title','authors','genres']])


rec_df = pd.DataFrame(columns=['title','authors','genres','average_rating','similarity_score'])


for idx,score in sorted_similarity[1:to_choose]:
  if idx not in book_ids:
    row = data_mini.iloc[[idx]][['title','authors','genres','average_rating']]
    row['similarity_score'] = score
    rec_df = pd.concat([rec_df,row],ignore_index = True)

rec_df = rec_df.sort_values(by='average_rating',ascending=False)

print("recommended books")

'''for i in range(to_show):
  print(list(rec_df.iloc[i][['title','authors','genres','average_rating']]))'''

alpha = 0.6 #giving weight to the rating

rec_df['final_score'] = rec_df['average_rating']*alpha + rec_df['similarity_score']*(1-alpha)
rec_df.sort_values(by = 'final_score',ascending = False)


'''for idx,score in sorted_similarity[1:to_choose]:
  if idx not in book_ids:
    print('score: ',score,data_mini.iloc[idx]['title'])'''



(384,)
loved books
title      A Storm of Swords (A Song of Ice and Fire, #3)
authors                            ['George R.R. Martin']
genres                                 [fantasy, fiction]
Name: 121, dtype: object
recommended books


/tmp/ipython-input-3833232677.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rec_df = pd.concat([rec_df,row],ignore_index = True)


"for idx,score in sorted_similarity[1:to_choose]:\n  if idx not in book_ids:\n    print('score: ',score,data_mini.iloc[idx]['title'])"

In [ ]:
rec_df.head()

,title,authors,genres,average_rating,similarity_score,final_score
31,"The Way of Kings (The Stormlight Archive, #1)",['Brandon Sanderson'],"[fantasy, fiction]",4.64,0.519034,2.991614
11,A Song of Ice and Fire (A Song of Ice and Fire...,['George R.R. Martin'],"[fantasy, fiction, science-fiction]",4.63,0.610994,3.022398
3,A Song of Ice and Fire (A Song of Ice and Fire...,['George R.R. Martin'],"[fantasy, fiction, science-fiction]",4.63,0.686518,3.052607
14,BookRags Summary: A Storm of Swords,['BookRags'],"[fantasy, fiction]",4.59,0.596638,2.992655
18,"Empire of Storms (Throne of Glass, #5)",['Sarah J. Maas'],"[fantasy, young-adult, romance, fiction, paran...",4.58,0.555718,2.970287


In [ ]:
alpha = 0.6 #giving weight to the rating

rec_df['final_score'] = rec_df['average_rating']*alpha + rec_df['similarity_score']*(1-alpha)

In [ ]:
rec_df.sort_values(by = 'final_score',ascending = False)

,title,authors,genres,average_rating,similarity_score,final_score
3,A Song of Ice and Fire (A Song of Ice and Fire...,['George R.R. Martin'],"[fantasy, fiction, science-fiction]",4.63,0.686518,3.052607
11,A Song of Ice and Fire (A Song of Ice and Fire...,['George R.R. Martin'],"[fantasy, fiction, science-fiction]",4.63,0.610994,3.022398
0,A Game of Thrones / A Clash of Kings (A Song o...,['George R.R. Martin'],"[fantasy, fiction, science-fiction]",4.53,0.721857,3.006743
7,A Storm of Swords: Blood and Gold (A Song of I...,['George R.R. Martin'],"[fantasy, fiction]",4.56,0.666678,3.002671
14,BookRags Summary: A Storm of Swords,['BookRags'],"[fantasy, fiction]",4.59,0.596638,2.992655
31,"The Way of Kings (The Stormlight Archive, #1)",['Brandon Sanderson'],"[fantasy, fiction]",4.64,0.519034,2.991614
18,"Empire of Storms (Throne of Glass, #5)",['Sarah J. Maas'],"[fantasy, young-adult, romance, fiction, paran...",4.58,0.555718,2.970287
4,A Storm of Swords: Steel and Snow (A Song of I...,['George R.R. Martin'],"[fantasy, fiction]",4.49,0.684063,2.967625
1,"A Game of Thrones (A Song of Ice and Fire, #1)",['George R.R. Martin'],"[fantasy, fiction]",4.45,0.692944,2.947178
9,"A Game of Thrones: The Graphic Novel, Vol. 1","['[Daniel Abraham', 'George R.R. Martin', 'Tom...","[fantasy, graphic-novels, comics, fiction, man...",4.48,0.640482,2.944193


In [ ]:

similarity3_list = list(enumerate(similarity3))

sorted_similarity = sorted(similarity3_list,key = lambda x:x[1],reverse = True)


In [ ]:
print(sorted_similarity)

[(121, array([1.0000001], dtype=float32)), (3476, array([0.72185713], dtype=float32)), (34, array([0.6929439], dtype=float32)), (170, array([0.6892624], dtype=float32)), (2362, array([0.68651795], dtype=float32)), (1875, array([0.68406296], dtype=float32)), (5699, array([0.67846334], dtype=float32)), (100, array([0.6761476], dtype=float32)), (1406, array([0.6666776], dtype=float32)), (9173, array([0.65823114], dtype=float32)), (8399, array([0.6404821], dtype=float32)), (5591, array([0.6204605], dtype=float32)), (2016, array([0.61099386], dtype=float32)), (3672, array([0.61042416], dtype=float32)), (149, array([0.60201174], dtype=float32)), (4252, array([0.5966381], dtype=float32)), (2151, array([0.586222], dtype=float32)), (5191, array([0.5816726], dtype=float32)), (8319, array([0.57800364], dtype=float32)), (2031, array([0.5557175], dtype=float32)), (7229, array([0.5550471], dtype=float32)), (6217, array([0.54934096], dtype=float32)), (3228, array([0.5488131], dtype=float32)), (4080, 

In [ ]:
data_mini.iloc[63]['title']

"Ender's Game (Ender's Saga, #1)"

In [ ]:

to_choose = 50

#print the loved books
print("loved books")
for book_id in book_ids:
  print(data_mini.iloc[book_id][['title','authors','genres']])


print("recommended books")
for idx,score in sorted_similarity[1:to_choose]:
  if idx not in book_ids:
    print('score: ',score,data_mini.iloc[idx]['title'])




loved books
title      A Storm of Swords (A Song of Ice and Fire, #3)
authors                            ['George R.R. Martin']
genres                                 [fantasy, fiction]
Name: 121, dtype: object
recommended books
score:  [0.72185713] A Game of Thrones / A Clash of Kings (A Song of Ice and Fire, #1-2)
score:  [0.6929439] A Game of Thrones (A Song of Ice and Fire, #1)
score:  [0.6892624] A Dance with Dragons (A Song of Ice and Fire, #5)
score:  [0.68651795] A Song of Ice and Fire (A Song of Ice and Fire, #1-5)
score:  [0.68406296] A Storm of Swords: Steel and Snow (A Song of Ice and Fire, #3: Part 1 of 2)
score:  [0.67846334] A Dance with Dragons: After the Feast (A Song of Ice and Fire #5, Part 2 of 2)
score:  [0.6761476] A Clash of Kings  (A Song of Ice and Fire, #2)
score:  [0.6666776] A Storm of Swords: Blood and Gold (A Song of Ice and Fire, #3: Part 2 of 2)
score:  [0.65823114] The World of Ice & Fire: The Untold History of Westeros and the Game of Thrones
score:  [

In [ ]:
data_mini.head()

,title,authors,average_rating,ratings_count,genres,description,image_url
0,"The Hunger Games (The Hunger Games, #1)",['Suzanne Collins'],4.34,4780653,"[young-adult, fiction, fantasy, science-fictio...",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,https://images.gr-assets.com/books/1447303603m...
1,Harry Potter and the Sorcerer's Stone (Harry P...,"['J.K. Rowling', 'Mary GrandPré']",4.44,4602479,"[fantasy, fiction, young-adult, classics]",Harry Potter's life is miserable. His parents ...,https://images.gr-assets.com/books/1474154022m...
2,"Twilight (Twilight, #1)",['Stephenie Meyer'],3.57,3866839,"[young-adult, fantasy, romance, fiction, paran...",About three things I was absolutely positive.\...,https://images.gr-assets.com/books/1361039443m...
3,To Kill a Mockingbird,['Harper Lee'],4.25,3198671,"[classics, fiction, historical-fiction, young-...",The unforgettable novel of a childhood in a sl...,https://images.gr-assets.com/books/1361975680m...
4,The Great Gatsby,['F. Scott Fitzgerald'],3.89,2683664,"[classics, fiction, historical-fiction, romance]",Alternate Cover Edition ISBN: 0743273567 (ISBN...,https://images.gr-assets.com/books/1490528560m...


So far, it works fairly well.

But the model we used all-miniLM-l6-v2 is, while small and convenient, is not very accurate. So, I wanted to use a better model that captures the meaning more accurately.

That model is - 'all-mpnet-base-v2'

Now, instead of using MHE for genres and authors and then combining with the description embeddings, it's possible to get a comprehensive description that includes authors and genres. This lets us avoid using MHE altogether and gives rich embeddings overall.


In [ ]:
import ast
import re
import pandas as pd

def safe_list(value):
    """Convert stringified lists or messy author/genre fields into clean list of strings."""
    if isinstance(value, list):
        return [str(v).strip(" []'\"") for v in value]
    elif isinstance(value, str):
        # Try to parse stringified list
        try:
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list):
                return [str(v).strip(" []'\"") for v in parsed]
        except Exception:
            pass
        # Fallback: split by comma
        return [v.strip(" []'\"") for v in value.split(',')]
    elif pd.isna(value):
        return []
    else:
        return [str(value).strip(" []'\"")]

def make_modified_description(row):
    title = str(row.get('title', '') or '').strip()
    authors = safe_list(row.get('authors', []))
    genres = safe_list(row.get('genres', []))
    description = str(row.get('description', '') or '').strip()

    authors_text = ', '.join(authors)
    genres_text = ', '.join(genres)

    # Compose the clean structured text
    combined = (
        f"Book Title: {title}. "
        f"Authors: {authors_text}. "
        f"Genres: {genres_text}. "
        f"Description: {description}"
    )
    return combined


In [ ]:
#data_mini = data_mini.copy()

data_mini.loc[:, 'combined_text'] = data_mini.apply(make_modified_description, axis=1)


In [ ]:
#saving the data with combined feature description

data_mini.to_csv("/content/drive/MyDrive/data science practice/book_rec_system/data_mini_books_update.csv",index=False)

In [ ]:
print(data_mini['combined_text'].iloc[9997])

Book Title: Breaking Point (Tom Clancy's Net Force, #4). Authors: Steve Perry, Tom Clancy, Steve Pieczenik. Genres: fiction, thriller, books, mystery, suspense, science-fiction. Description: In the year 2000, computers are the new superpowers. Those who control them control the world. To enforce the Net Laws, Congress creates the ultimate computer security agency within the FBI: the Net Force.Reeling from a shattered personal life, Net Force Commander Alex Michaels is informed that top secret information from a joint Air Force--Navy venture has been accessed and downloaded. The research involves an atmospheric weapon with the capability to drive half a country into madness using low frequency wave generation. Now the technology has fallen into the wrong hands--and testing has begun...


So far, we got to see the combined description that includes title, author, genres along with book description.

Next, load the model

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2',device = 'cuda')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#now we get the embeddings using the model


data_mini = pd.read_csv("/content/drive/MyDrive/data science practice/book_rec_system/data_mini_books_update.csv")

book_texts = data_mini['combined_text'].tolist()
book_embeddings = model.encode(book_texts, normalize_embeddings=True,batch_size = 64,convert_to_numpy = True,show_progress_bar = True)

import numpy as np
#np.save("/content/drive/MyDrive/data science practice/book_rec_system/book_embeddings.npy", book_embeddings)

book_embeddings = np.load("/content/drive/MyDrive/data science practice/book_rec_system/book_embeddings.npy")


NameError: name 'model' is not defined

In [ ]:
data_mini.head(10)

,title,authors,average_rating,ratings_count,genres,description,image_url,popularity,combined_text
index,,,,,,,,,
0,"The Hunger Games (The Hunger Games, #1)",['Suzanne Collins'],4.34,4780653,"[young-adult, fiction, fantasy, science-fictio...",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,https://images.gr-assets.com/books/1447303603m...,0.897872,Book Title: The Hunger Games (The Hunger Games...
1,Harry Potter and the Sorcerer's Stone (Harry P...,"['J.K. Rowling', 'Mary GrandPré']",4.44,4602479,"[fantasy, fiction, young-adult, classics]",Harry Potter's life is miserable. His parents ...,https://images.gr-assets.com/books/1474154022m...,0.900503,Book Title: Harry Potter and the Sorcerer's St...
2,"Twilight (Twilight, #1)",['Stephenie Meyer'],3.57,3866839,"[young-adult, fantasy, romance, fiction, paran...",About three things I was absolutely positive.\...,https://images.gr-assets.com/books/1361039443m...,0.638414,"Book Title: Twilight (Twilight, #1). Authors: ..."
3,To Kill a Mockingbird,['Harper Lee'],4.25,3198671,"[classics, fiction, historical-fiction, young-...",The unforgettable novel of a childhood in a sl...,https://images.gr-assets.com/books/1361975680m...,0.713173,Book Title: To Kill a Mockingbird. Authors: Ha...
4,The Great Gatsby,['F. Scott Fitzgerald'],3.89,2683664,"[classics, fiction, historical-fiction, romance]",Alternate Cover Edition ISBN: 0743273567 (ISBN...,https://images.gr-assets.com/books/1490528560m...,0.582683,Book Title: The Great Gatsby. Authors: F. Scot...
5,The Fault in Our Stars,['John Green'],4.26,2346404,"[young-adult, romance, fiction, contemporary]",Despite the tumor-shrinking medical miracle th...,https://images.gr-assets.com/books/1360206420m...,0.626113,Book Title: The Fault in Our Stars. Authors: J...
6,The Hobbit,['J.R.R. Tolkien'],4.25,2071616,"[fantasy, classics, fiction, young-adult]",In a hole in the ground there lived a hobbit. ...,https://images.gr-assets.com/books/1372847500m...,0.595229,Book Title: The Hobbit. Authors: J.R.R. Tolkie...
7,The Catcher in the Rye,['J.D. Salinger'],3.79,2044241,"[classics, fiction, young-adult]",The hero-narrator of The Catcher in the Rye is...,https://images.gr-assets.com/books/1398034300m...,0.494492,Book Title: The Catcher in the Rye. Authors: J...
8,"Angels & Demons (Robert Langdon, #1)",['Dan Brown'],3.85,2001311,"[fiction, mystery, thriller, suspense, crime, ...",World-renowned Harvard symbologist Robert Lang...,https://images.gr-assets.com/books/1303390735m...,0.502765,"Book Title: Angels & Demons (Robert Langdon, ..."


# 2nd recommendation




In [ ]:

from sklearn.metrics.pairwise import cosine_similarity


book_ids = [5780,5900,6789,7899]

#get average of the book_ids embeds
avg_book_embeds = sum(book_embeddings[book_ids])/len(book_ids)
avg_book_embeds.reshape(1,-1)
print(avg_book_embeds.shape)



similarity3 = cosine_similarity(book_embeddings, avg_book_embeds.reshape(1, -1))
#print(similarity3.shape)
similarity3_list = list(enumerate(similarity3))
sorted_similarity = sorted(similarity3_list,key = lambda x:x[1],reverse = True)

to_choose = 50
to_show = 15

#print the loved books
print("loved books")
for book_id in book_ids:
  print(data_mini.iloc[book_id][['title','authors','genres']])


rec_df = pd.DataFrame(columns=['title','authors','genres','average_rating','similarity_score'])


for idx,score in sorted_similarity[1:to_choose]:
  if idx not in book_ids:
    row = data_mini.iloc[[idx]][['title','authors','genres','average_rating']]
    row['similarity_score'] = score
    rec_df = pd.concat([rec_df,row],ignore_index = True)

rec_df = rec_df.sort_values(by='average_rating',ascending=False)

print("recommended books")

'''for i in range(to_show):
  print(list(rec_df.iloc[i][['title','authors','genres','average_rating']]))'''

alpha = 0.3 #giving weight to the rating

rec_df['final_score'] = rec_df['average_rating']*alpha + rec_df['similarity_score']*(1-alpha)
rec_df.sort_values(by = 'final_score',ascending = False)




NameError: name 'book_embeddings' is not defined

Finally, we can see the recommendations given by the model.

So, this is a content based recommendation system for books.